# Lesson 11: Linear Regression

We aim to predict the life expenctancy of different countries during different years using socioeconomic features.  

Linear regresssion is **not compatible** with missing values or categorical variables. 

## Imports

In [2]:
# This is to change the top level directory.
%cd ..

/Users/jaimemerizalde/Desktop/Library/Machine Learning Udemy Course


In [ ]:
from Library import data
import pandas as pd

pd.set_option("display.max_columns", None)

# Get Data

In [8]:
df = data.get_data("Datasets/LifeExpectancy.csv", index_col=[0])
df.shape

(2938, 22)

There are 22 columns / features and 2938 rows of data.

# Preprocessing

Removing missing values and categorical variables. They will not work with a linear model.  

## Missing Values

Check how many missing values.  
This will give us how many missing values, from most to least, there are in this datset.

In [ ]:
df.isna().sum().sort_values(ascending=False)

population                         652
hepatitis_b                        553
gdp                                448
total_expenditure                  226
alcohol                            194
income_composition_of_resources    167
schooling                          163
thinness_5_9_years                  34
thinness__1_19_years                34
bmi                                 34
polio                               19
diphtheria                          19
life_expectancy                     10
adult_mortality                     10
hiv_aids                             0
country                              0
year                                 0
measles                              0
percentage_expenditure               0
infant_deaths                        0
status                               0
under_five_deaths                    0
dtype: int64

## Categorial Variables

In [24]:
categorical_columns = list(df.dtypes[df.dtypes == "O"].index.values)
categorical_columns

# Recast non-numerical/object columns as CAT_VARS.
for column in categorical_columns:
    df[column] = df[column].astype("category")

df.dtypes

country                            category
year                                  int64
status                             category
life_expectancy                     float64
adult_mortality                     float64
infant_deaths                         int64
alcohol                             float64
percentage_expenditure              float64
hepatitis_b                         float64
measles                               int64
bmi                                 float64
under_five_deaths                     int64
polio                               float64
total_expenditure                   float64
diphtheria                          float64
hiv_aids                            float64
gdp                                 float64
population                          float64
thinness__1_19_years                float64
thinness_5_9_years                  float64
income_composition_of_resources     float64
schooling                           float64
dtype: object